In [89]:
%reset -f
from parflow import Run
from parflow.tools.fs import get_absolute_path
from parflow.tools.io import write_pfb, read_pfb
import numpy as np
from matplotlib import pyplot as plt
# from parflow.tools.hydrology import calculate_water_table_depth, \
# calculate_overland_flow_grid, calculate_overland_fluxes
# %matplotlib widget

In [90]:
nx = 20
ny = 5
nz = 20
dx = 5
dy = 0.2
dz = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, \
               0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.3, 0.1])

ztemp = np.cumsum(dz) - dz

print(ztemp)

[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6.  6.5 7.  7.5 8.  8.5
 9.  9.3]


In [91]:
Vx = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velx.43800.pfb'))
Vy = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.vely.43800.pfb'))
Vz = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velz.43800.pfb'))
Saturation = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.satur.43800.pfb'))
Porosity = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.porosity.pfb'))
# print(Vx.shape, Vy.shape, Vz.shape)

In [92]:
#setup indices

indx = np.zeros((nz,ny,nx))
for i in range(nx):  
    indx[:,:,i] = np.ones((nz,ny)) * i
# print(indx[10,:,:])

indy = np.zeros((nz,ny,nx))
for j in range(ny):  
    indy[:,j,:] = np.ones((nz,nx)) * j
# print(indy[10,:,:])

indz = np.zeros((nz,ny,nx))
for k in range(nz):  
    indz[k,:,:] = np.ones((ny,nx)) * k
# print(indz[10,:,:])

In [93]:
#setup initial location
npcell = 40
# how many particles in a grid cell

initx = np.zeros((npcell,nz,ny,nx))
inity = np.zeros((npcell,nz,ny,nx))
initz = np.zeros((npcell,nz,ny,nx))

# generate random numbers
# uniform distribution between 0 and 1 with mean of 0.5 and std of 1/(2*sqrt(3))

for i in range(npcell):
    randx = np.random.uniform(low=0.0, high=1.0, size=(nz,ny,nx))
    randy = np.random.uniform(low=0.0, high=1.0, size=(nz,ny,nx))
    randz = np.random.uniform(low=0.0, high=1.0, size=(nz,ny,nx))
    initx[i,:,:,:] = (indx + randx)*dx
    inity[i,:,:,:] = (indy + randy)*dy
    initz[i,:,:,:] = randz*dz[:, np.newaxis, np.newaxis] + \
                     ztemp[:, np.newaxis, np.newaxis]
# initz[1,0,:,:]

In [94]:
#cal locid
Px = np.copy(initx)
Py = np.copy(inity)
Pz = np.copy(initz)

Plocx = np.floor(Px/dx)
Plocy = np.floor(Py/dy)

# Plocz = np.where(ztemp == Pz[:,:,:,:,None])[4]
# Plocz = Plocz.reshape(npcell,nz,ny,nx)
Plocz = np.zeros((npcell,nz,ny,nx))

for ii in range(npcell):
    for i in range(nz):
        for j in range(ny):
            for k in range(nx):
                zz = 0
                for ll in range(nz):
                    zz = zz + dz[ll]
                    if zz >= Pz[ii,i,j,k]:
                        Plocz[ii,i,j,k] = ll
                        break            

In [95]:
# cal vel loc
# this can be got by random number directly
# but for multiple steps, we still need to calculate
# so we provide the template here
# we may also calculate it from random number
# i mean like add the intial fraction with the new_distance/vel
# but additional other calculation may need

Clocx = (Px - Plocx*dx)/dx
Clocy = (Py - Plocy*dy)/dy
Clocz = np.zeros((npcell,nz,ny,nx))

for ii in range(npcell):
    for i in range(nz):
        for j in range(ny):
            for k in range(nx):
                zz = 0
                temp = int(Plocz[ii,i,j,k])
                if temp > 0:
                    zz = np.sum(dz[0:temp])
                Clocz[ii,i,j,k] = (Pz[ii,i,j,k] - zz)/dz[temp]

In [96]:
Vpx = np.zeros((npcell,nz,ny,nx))
Vpy = np.zeros((npcell,nz,ny,nx))
Vpz = np.zeros((npcell,nz,ny,nx))

for ii in range(npcell):
    for i in range(nz):
        for j in range(ny):
            for k in range(nx):
                
                tempx = int(Plocx[ii,i,j,k])
                tempy = int(Plocy[ii,i,j,k])
                tempz = int(Plocz[ii,i,j,k])
                    
                Vpx[ii,i,j,k] = ((1 - Clocx[ii,i,j,k])*Vx[tempz,tempy,tempx] + \
                                Clocx[ii,i,j,k]*Vx[tempz,tempy,tempx+1])/ \
                                Saturation[tempz,tempy,tempx]/Porosity[tempz,tempy,tempx]
                
                Vpy[ii,i,j,k] = ((1 - Clocy[ii,i,j,k])*Vy[tempz,tempy,tempx] + \
                                Clocy[ii,i,j,k]*Vy[tempz,tempy+1,tempx])/ \
                                Saturation[tempz,tempy,tempx]/Porosity[tempz,tempy,tempx]
                
                Vpz[ii,i,j,k] = ((1 - Clocz[ii,i,j,k])*Vz[tempz,tempy,tempx] + \
                                Clocz[ii,i,j,k]*Vz[tempz+1,tempy,tempx])/ \
                                Saturation[tempz,tempy,tempx]/Porosity[tempz,tempy,tempx]

In [97]:
Px = Px + Vpx
Py = Py + Vpy
Pz = Pz + Vpz

In [98]:
import torch
from torch.utils.data import TensorDataset

boundings = np.array([0., 100., 0., 1., 0., 9.4])
minval = -1
maxval = 1
i = 1
interval = 1
step_size = 1
t_start = 0
t_end = 10

Px = (Px - boundings[0]) / (boundings[1] - boundings[0]) * (maxval - minval) +  minval
Py = (Py - boundings[2]) / (boundings[3] - boundings[2]) * (maxval - minval) +  minval
Pz = (Pz - boundings[4]) / (boundings[5] - boundings[4]) * (maxval - minval) +  minval

initx = (initx - boundings[0]) / (boundings[1] - boundings[0]) * (maxval - minval) +  minval
inity = (inity - boundings[2]) / (boundings[3] - boundings[2]) * (maxval - minval) +  minval
initz = (initz - boundings[4]) / (boundings[5] - boundings[4]) * (maxval - minval) +  minval

t = (i * interval * step_size - t_start) / (t_end - t_start) * (maxval - minval) +  minval ## start time   

start = torch.zeros(npcell*nz*ny*nx,3)
end   = torch.zeros(npcell*nz*ny*nx,3)
time  = torch.zeros(npcell*nz*ny*nx,1)

count = 0
for ii in range(npcell):
    for i in range(nz):
        for j in range(ny):
            for k in range(nx):  
                
                start[count,:,] = torch.FloatTensor([initx[ii,i,j,k],inity[ii,i,j,k],initz[ii,i,j,k]])
                end[count,:]    = torch.FloatTensor([Px[ii,i,j,k],Py[ii,i,j,k],Pz[ii,i,j,k]])
                # time[count]     = torch.FloatTensor(np.array([t]).reshape(-1,1))
                time[count]     = torch.FloatTensor([t]).reshape(-1,1)
                count           = count + 1

In [99]:
data = TensorDataset(start,end,time)
torch.save(data,'data.pth')

In [100]:
count = 0
for x in data:#generator
    print(x)
    count = count + 1
    if count > 100: break

(tensor([-0.9173, -0.6983, -0.8947]), tensor([-0.9174, -0.6983, -0.8947]), tensor([-0.8000]))
(tensor([-0.8658, -0.7437, -0.9387]), tensor([-0.8659, -0.7437, -0.9387]), tensor([-0.8000]))
(tensor([-0.7006, -0.8168, -0.8957]), tensor([-0.7007, -0.8168, -0.8957]), tensor([-0.8000]))
(tensor([-0.6837, -0.7101, -0.9276]), tensor([-0.6838, -0.7101, -0.9276]), tensor([-0.8000]))
(tensor([-0.5306, -0.8171, -0.9151]), tensor([-0.5307, -0.8171, -0.9151]), tensor([-0.8000]))
(tensor([-0.4261, -0.9108, -0.9358]), tensor([-0.4262, -0.9108, -0.9358]), tensor([-0.8000]))
(tensor([-0.3613, -0.7083, -0.9005]), tensor([-0.3614, -0.7083, -0.9005]), tensor([-0.8000]))
(tensor([-0.2651, -0.7945, -0.9038]), tensor([-0.2652, -0.7945, -0.9039]), tensor([-0.8000]))
(tensor([-0.1263, -0.7133, -0.9602]), tensor([-0.1264, -0.7133, -0.9602]), tensor([-0.8000]))
(tensor([-0.0202, -0.8562, -0.9074]), tensor([-0.0203, -0.8562, -0.9074]), tensor([-0.8000]))
(tensor([ 0.0173, -0.9313, -0.9578]), tensor([ 0.0172, -0.93